In [ ]:
from __future__ import print_function
from music21 import *
from collections import Counter, defaultdict

import initialisation
import modelTrainer
import musicPlayer
import simulation
import fitness
import collections
import importlib
import music21
import numpy as np
import pandas as pd
import individual
import duration
import constants
import util
import random
import nltk
import fitness
import re


In [ ]:
notes = ['C#4', 'D4', 'E4']
map(lambda x: re.sub('\d+', '', x)

In [52]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(fitness)
importlib.reload(music21)
importlib.reload(constants)
importlib.reload(util)
importlib.reload(individual)
importlib.reload(fitness)

<module 'fitness' from 'B:\\School\\Master\\Thesis\\MasterThesis\\EvoMusicCompanion\\fitness.py'>

In [ ]:
len(scores)

## Get corpus

In [ ]:
coreCorpus = music21.corpus.corpora.CoreCorpus()
curr_corpus = music21.corpus.search('mozart', 'composer')

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
lakh_corpus = corpus.corpora.LocalCorpus('lakh')
curr_corpus = lakh_corpus.metadataBundle[0:20]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
wiki_corpus = wiki_corpus.metadataBundle[0:20]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
score = music21.corpus.parse('joplin/maple_leaf_rag.mxl')
scores = util.transpose_piece(score, 'C')
scores = [scores]

In [ ]:
scores[0].show('musicxml')

In [ ]:
dur_matrix = modelTrainer.train_duration_matrix(scores)
pitch_matrix = modelTrainer.train_pitch_matrix(scores);

In [ ]:
NOTE_RANGE = [
    'A4',
    'A#4',
    'B4',
    'C4',
    'C#4',
    'D4',
    'D#4',
    'E4',
    'F4',
    'G4',
    'G#4',
    'A5',
    'A#5',
    'B5',
    'B#5',
    'C#5',
    'D5',
    'D#5',
    'E5',
    'F5',
    'G5',
    'G#5',
]

In [ ]:
# Interval matrix
first_notes_per_bar = []
all_intervals = []
for s in scores:
    noteIterator = s.parts[0].getElementsByClass(stream.Measure)
    if len(noteIterator) == 0:
        noteIterator = s.parts[0].notesAndRests.stream()
    for i in noteIterator:
        firstNote = None
        counter = 1
        m = []
        for j in i.notesAndRests:
            if j.isChord:
                curr_note = note.Note(j.root())
            elif j.isRest:
                m.append('REST')
                continue
            else:
                curr_note = j
            if counter == 1:
                if j.isChord:
                    firstNote = note.Note(j.root())
                else:
                    firstNote = j
                pitch_name = firstNote.nameWithOctave
                if '-' in pitch_name or '##' in pitch_name:
                    pitch_name = firstNote.pitch.getEnharmonic().nameWithOctave
                if pitch_name in NOTE_RANGE:
                    first_notes_per_bar.append(pitch_name)
                counter += 1
                continue
            #print(j.beatStr)
            octave = curr_note.octave
            root = note.Note('C')
            root.octave = octave
            interv = interval.Interval(root,  curr_note).name
            m.append(interv)
            counter += 1
        all_intervals.append(interv)
all_intervals

bigrams = list(nltk.bigrams(all_intervals))
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for i1,i2 in bigrams:
    matrix[i1][i2] += 1
    
for i1 in matrix:
    total = float(sum(matrix[i1].values()))
    for i2 in matrix[i1]:
        matrix[i1][i2] /= total
    
int_matrix = pd.DataFrame(matrix)
int_matrix = int_matrix.fillna(0)


In [ ]:
trigrams = list(nltk.trigrams(all_intervals))
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for i1,i2,i3 in trigrams:
    matrix[(i1,i2)][i3] += 1
    
for i1 in matrix:
    total = float(sum(matrix[i1].values()))
    for i2 in matrix[i1]:
        matrix[i1][i2] /= total
    
int_trigram_matrix = pd.DataFrame(matrix)
int_trigram_matrix = int_matrix.fillna(0)

In [51]:
c = chord.Chord(['E3', 'F3', 'G3'])
c.root()

<music21.pitch.Pitch E3>

In [ ]:
init_note_vector = defaultdict(lambda: 0)
for i in first_notes_per_bar:
    init_note_vector[i] += 1
total = float(sum(init_note_vector.values()))
for i in init_note_vector:
    init_note_vector[i] /= total
init_note_vector = pd.Series(init_note_vector)

In [ ]:
rng = random.random()
if start_duration is None:
    start_duration = random.choice(dur_matrix.keys())

transitions = dur_matrix[start_duration]
p_sum = 0.0

for k, v in transitions.iteritems():
    if p_sum < rng < p_sum + v:
        print(k)
    else:
        p_sum += v

In [ ]:
population

In [53]:
sim = simulation.Simulation(0.5, 100)
population = sim.run_interactively(pitch_matrix, dur_matrix, None)

Starting generation


In [54]:
sim.run(10, pitch_matrix, dur_matrix)

In [ ]:
sim.pitch_matrix

In [ ]:
chord.Chord(['C3', 'E3', 'G3'], quarterlength='4')

In [55]:
musicPlayer.play(population[0:4])

key = C major


KeyboardInterrupt: 

In [ ]:
selection = [population[0], population[1]]
selection

In [ ]:
selected_population_notes = constants.flatten(list(map(lambda x: x.notes, selection)))
selected_population_pitches = list(map(lambda x: x.pitch, constants.flatten(selected_population_notes)))
selected_population_pitches

In [ ]:
population = sim.update(selection=selection)

In [ ]:
[i for sublist in [[[n.pitch for n in m.notes] for m in p.measures] for p in population[0:1]] for i in sublist]

for p in population:
    for m in p.measures:
        for n in p.notes:
            print(n.pitch)

In [ ]:
[population[1].measures[2].notes, list(map(lambda x: x.notes, population[5].measures[1:3]))]

In [46]:
musicPlayer.play(population)

key = C major
